# Get forecast for a specific spot

In [191]:
from pysurfline import SpotForecast

In [192]:
params={
    "spotId":"584204204e65fad6a7709b09",
    "days":7,
    "intervalHours":3,
    }
spot=SpotForecast(params,verbose=True)

-----
ForecastGetter(Type:wave, Status:200)
-----
ForecastGetter(Type:wind, Status:200)
-----
ForecastGetter(Type:tides, Status:200)
-----
ForecastGetter(Type:weather, Status:200)


In [193]:
waves=spot.get_dataframe("wave")
waves.head()

,utcOffset,surf_min,surf_max,surf_optimalScore,swells
timestamp,,,,,
2022-02-12 23:00:00,1,0.0,0.16,0,"[{'height': 0.89362, 'period': 4, 'direction':..."
2022-02-13 02:00:00,1,0.0,0.18,0,"[{'height': 0.67722, 'period': 4, 'direction':..."
2022-02-13 05:00:00,1,0.0,0.20,0,"[{'height': 0.49634, 'period': 3, 'direction':..."
2022-02-13 08:00:00,1,0.0,0.18,0,"[{'height': 0.35984, 'period': 3, 'direction':..."
2022-02-13 11:00:00,1,0.0,0.16,0,"[{'height': 0.23654, 'period': 3, 'direction':..."


In [194]:
wind=spot.get_dataframe("wind")
wind.head()

,utcOffset,speed,direction,directionType,gust,optimalScore
timestamp,,,,,,
2022-02-12 23:00:00,1,21.38013,332.07711,Offshore,31.69228,2
2022-02-13 02:00:00,1,19.51271,333.45029,Offshore,26.65024,2
2022-02-13 05:00:00,1,16.29204,335.93689,Offshore,20.54347,2
2022-02-13 08:00:00,1,14.55090,335.78409,Offshore,18.36000,2
2022-02-13 11:00:00,1,10.13007,351.38864,Offshore,11.88953,0


In [195]:
tides=spot.get_dataframe("tides")
tides.head()

,utcOffset,type,height
timestamp,,,
2022-02-12 23:00:00,1,NORMAL,0.13
2022-02-12 23:45:38,1,LOW,0.13
2022-02-13 00:00:00,1,NORMAL,0.13
2022-02-13 01:00:00,1,NORMAL,0.13
2022-02-13 02:00:00,1,NORMAL,0.12


In [196]:
weather=spot.get_dataframe("weather")
weather.head()

,utcOffset,temperature,condition
timestamp,,,
2022-02-12 23:00:00,1,6.55508,NIGHT_CLEAR
2022-02-13 02:00:00,1,6.29089,NIGHT_CLEAR
2022-02-13 05:00:00,1,5.88278,NIGHT_CLEAR
2022-02-13 08:00:00,1,6.50375,CLEAR
2022-02-13 11:00:00,1,9.39313,CLEAR
